# **Pretraing GPT-2**

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
   

In [ ]:
from datasets import load_dataset
dataset = load_dataset("Trelis/tiny-shakespeare")
# dataset load_dataset('csv',)
def preprocess_function(examples):
    return tokenizer(examples['Text'], truncation=True, padding='max_length')
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("gpt2")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Step,Training Loss
10,3.518700
20,3.409600
30,3.419800
40,3.433700
50,3.309300
60,3.242200
70,3.322300
80,3.248400
90,3.152700
100,3.181500


TrainOutput(global_step=590, training_loss=3.048778278544798, metrics={'train_runtime': 876.7531, 'train_samples_per_second': 2.692, 'train_steps_per_second': 0.673, 'total_flos': 1233298391040000.0, 'train_loss': 3.048778278544798, 'epoch': 5.0})

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub("ahmadmac/Pretrained-GPT2")
tokenizer.push_to_hub("ahmadmac/Pretrained-GPT2")

README.md:   0%|          | 0.00/993 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ahmadmac/Pretrained-GPT2/commit/7c642d76babedec66fdc925b1bdf0113c192ee09', commit_message='Upload tokenizer', commit_description='', oid='7c642d76babedec66fdc925b1bdf0113c192ee09', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("ahmadmac/Pretrained-GPT2")
model = AutoModelForCausalLM.from_pretrained("ahmadmac/Pretrained-GPT2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
def get_response(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [ ]:
print(get_response("I have heard you say that"))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


I have heard you say that the Lord of the North is a man of great power, and that he hath made a great deal of peace with the people.

KING RICHARD II:
I have heard you say that the Lord of the North is a man of great power, and that he hath made a great deal of peace with the people.

KING RICHARD II:
I have heard you say that the Lord of the North is a man of great power,


In [ ]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    save_total_limit=2,
    save_steps=200,
    per_device_train_batch_size=4,
    save_strategy="epoch",
    logging_steps=10,
    logging_dir='./logs',
)